# Suitability

* High Volatility
* Strong Directional momentum
* Liquidity
* Low event risk

# Key factors and metrics

* Pre-market sentiment: if a stock is up pre-market, high change for it being bullish
* Trend indicators: Moving averages, momentum indicators, confirming the direction.
* Volatility indicators: Avg true range (higher means more daily movement), implied volatility (high usually means big moves or uncertainty)
* Volume confirmation: intraday volume should be above avg when the price moves in my direction
* News-driven catalysts: Earnings reports, product announcements, macroeconomic news.

# Practical workflow for evaluation
* Scan for candidate stocks with >2% pre-market move, high volume, and news.
* Charts, look for strong pre-market trends or breakouts on daily charts.
* Intraday confirmation: Wait for 30-60 mins to confirm direction.
* Entry and Exit levels: example, risk 2% downside for 4-6% upside >= 2:1 ratio.

# Reducing risk
* Never hold overnight: ETPs can gap heavilty the next day
* Use hard stop-loss: limit your losses to a fixed % (eg 5-10% of the leveraged ETP Price)
* Size small: use only 100-200$
* Avoid choppy/sideways stocks: decay will eat into returns.

# Automating evaluation via code

* Pull pre-market movers for your watchlist
* Calculate % change from previous close, Intraday volatility(ATR), Volume vs avg Volume.
* Apply filters: price > 20MA, pre-market >2%, volume > 150% of avg etc.
* Signal when a stocck is worth trading on leverage for that day.

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from ta.momentum import RSIIndicator
import pytz

# ----------------------------
# CONFIG
# ----------------------------
TICKERS = ["AMD", "GOOGL", "TSLA", "NVDA"]
LEVERAGED_ETPS = {
    "AMD": "3AMD.L",
    "GOOGL": "3GOO.L",
}
PREMARKET_GAP_THRESHOLD = 2.0
VOLUME_RATIO_THRESHOLD = 1.5
RSI_BUY_LEVEL = 50

# Timezone for market open/close checks (New York for US stocks)
ny_tz = pytz.timezone("America/New_York")

# ----------------------------
# FUNCTIONS
# ----------------------------

def get_premarket_gap(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="5d", interval="1d")
    if hist.empty:
        return None, None

    prev_close = hist['Close'].iloc[-1]

    intraday_hist = stock.history(period="1d", interval="1m")
    if intraday_hist.empty:
        return None, None

    current_price = intraday_hist['Close'].iloc[-1]
    gap_pct = ((current_price - prev_close) / prev_close) * 100
    return gap_pct, current_price


def get_intraday_signals(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="5d", interval="5m")
    if hist.empty:
        return None, None, None

    # Fix timezone mismatch: use UTC for comparison
    today_date = dt.datetime.now(tz=pytz.UTC).date()
    today = hist.loc[hist.index.date == today_date]

    if today.empty:
        return None, None, None

    avg_vol = hist['Volume'].mean()
    vol_ratio = today['Volume'].sum() / avg_vol

    rsi = RSIIndicator(today['Close']).rsi().iloc[-1] if len(today) > 14 else None
    last_price = today['Close'].iloc[-1]

    return vol_ratio, rsi, last_price


def evaluate_tickers():
    signals = []
    for ticker in TICKERS:
        gap, current_price = get_premarket_gap(ticker)
        vol_ratio, rsi, last_price = get_intraday_signals(ticker)

        if gap is None or vol_ratio is None:
            continue

        signal = {
            "Ticker": ticker,
            "PreMarketGap%": round(gap, 2),
            "VolRatio": round(vol_ratio, 2),
            "RSI": round(rsi, 2) if rsi else None,
            "LastPrice": round(last_price, 2) if last_price else None,
            "ETP": LEVERAGED_ETPS.get(ticker, "N/A"),
            "Candidate": (gap > PREMARKET_GAP_THRESHOLD) and
                         (vol_ratio > VOLUME_RATIO_THRESHOLD) and
                         (rsi and rsi > RSI_BUY_LEVEL)
        }
        signals.append(signal)

    return pd.DataFrame(signals)

# ----------------------------
# RUN
# ----------------------------
df_signals = evaluate_tickers()

print("\n---- Leveraged ETP Candidates ----\n")
print(df_signals if not df_signals.empty else "No signals (market closed or filters not met)")




---- Leveraged ETP Candidates ----

No signals (market closed or filters not met)
